[IRuby](https://github.com/sciruby/iruby) notebook to convert a COMNET/EDA XML from OpenStudio's Parametric Analysis Tool to a JSON file.

In [1]:
require 'active_support'
require 'active_support/core_ext'
require 'json'
require 'openstudio'
require 'win32ole'
require 'C:\openstudio-spreadsheet\lib\excel.rb'

true

In [2]:
# constants
END_USES = ['heating', 'cooling', 'lighting_interior', 'lighting_exterior', 'equipment_interior', 'equipment_exterior', 'fans', 'pumps', 'heat_rejection', 'humidification', 'heat_recovery', 'water_systems', 'refrigeration', 'generators']
FUEL_TYPES = ['electricity', 'gas', 'other_energy', 'district_cooling', 'district_heating'] #, 'water']
MONTHS = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
INDENT = '  '
XML_NAME = 'report.xml'

"report.xml"

In [3]:
module Outputs

  # convert xml to json
  # https://stackoverflow.com/questions/1530324/ruby-xml-to-json-converter
  def self.xml_to_json(path)
  
    # convert XSD to XML
    extname = File.extname(path)  
    case extname
    when '.xml'
      path = path   
    # when '.xsd'  
    #  path.gsub!(extname, '.xml')
    else
      puts 'ERROR: file extension must be XML' # or XSD'
    end
  
    # convert XML to JSON
    file = IO::read(path)
    string = Hash.from_xml(file).to_json
    IO::write(path.gsub(extname, '.json'), string)  
    
  end

  # osm json to new json
  def self.osm_outputs(base_dir = nil, prop_dir = nil)

    #TODO add error checking
    base_results_json_path = File.join(base_dir, 'run/results.json')
    prop_results_json_path = File.join(prop_dir, 'run/results.json')

    paths_hash = {}
    paths_hash['base'] = base_results_json_path
    paths_hash['prop'] = prop_results_json_path

    base_prop_hash = {}

    paths_hash.each do |key, val|
      
      puts key, val
      
      type = key
      file = File.read(val)
      data = JSON.parse(file)['XcelEDAReportingandQAQC']
#       puts type, data

      # hash of {'fuel_type1' => {'end_use1' => {'month' => monthly_data_array, 'year' => annual_total}, ...}}
      fuel_types_end_uses_consumption_demand_hash = {}

      FUEL_TYPES.each do |fuel_type|

        # hashes of {'end_use1' => {'month' => monthly_data_array, 'year' => annual_total}, ...}
        end_use_consumption_hash = {}
        end_use_demand_hash = {}

        END_USES.each do |end_use|

          # hashes of {'month' => monthly_data_array, 'year' => annual_total}

          monthly_consumption_array = []
          monthly_demand_array = []

          (1..12).each do |n|
            monthly_consumption_array << data["monthly_consumption_#{end_use}_#{fuel_type}_month_#{n}"]
            monthly_demand_array << data["monthly_demand_#{end_use}_#{fuel_type}_month_#{n}"]
          end

          consumption_hash = {}
          consumption_hash['month'] = monthly_consumption_array
          consumption_hash['year'] = data["monthly_consumption_#{end_use}_#{fuel_type}_month_year"] #sum
          end_use_consumption_hash[end_use] = consumption_hash

          demand_hash = {}      
          demand_hash['month'] = monthly_demand_array
          demand_hash['year'] = data["monthly_demand_#{end_use}_#{fuel_type}_month_year"] #max
          end_use_demand_hash[end_use] = demand_hash

        end

        # hash of {'consumption/demand' => {'end_use1' => {'month' => monthly_data_array, 'year' => annual_total}, ...}
        end_use_consumption_demand_hash = {}
        end_use_consumption_demand_hash['consumption'] = end_use_consumption_hash
        end_use_consumption_demand_hash['demand'] = end_use_demand_hash

        fuel_types_end_uses_consumption_demand_hash[fuel_type] = end_use_consumption_demand_hash

      end

      base_prop_hash[type] = fuel_types_end_uses_consumption_demand_hash

    end

    # convert hash to json and write to file
    base_prop_json = base_prop_hash.to_json
    File.open('outputs.json', 'w') { |f| f << base_prop_json }    

  end

  # convert PAT results.json to outputs.json
  def self.pat_outputs(path)
    
    file = File.read(path)
    data = JSON.parse(file)  

    alternatives_hash = {}

    alternatives = data['analysis']['alternatives']['alternative']
    alternatives.each do |alternative|

      name = alternative['name']
      # puts name

      results = alternative['results']

      # cash_flows = results['cash_flows']
      # annual = results['annual']
      
      # get monthly data
      monthly = results['monthly']
      monthly_consumption = monthly['consumption']
      monthly_demand = monthly['demand']

      # hash of {'fuel_type1' => {'end_use1' => {'month' => monthly_data_array, 'year' => annual_total}, ...}}
      fuel_types_end_uses_consumption_demand_hash = {}

      FUEL_TYPES.each do |fuel_type|

        # hashes of {'end_use1' => {'month' => monthly_data_array, 'year' => annual_total}, ...}
        end_use_consumption_hash = {}
        end_use_demand_hash = {}

        END_USES.each do |end_use|

          # hashes of {'month' => monthly_data_array, 'year' => annual_total}

          consumption_hash = {}
          monthly_consumption_array = monthly_consumption[end_use][fuel_type]['month']
          consumption_hash['month'] = monthly_consumption_array
          consumption_hash['year'] = monthly_consumption[end_use][fuel_type]['year'] #sum
          end_use_consumption_hash[end_use] = consumption_hash
          
          demand_hash = {}
          monthly_demand_array = monthly_demand[end_use][fuel_type]['month']
          demand_hash['month'] = monthly_demand_array
          demand_hash['year'] = monthly_demand[end_use][fuel_type]['year'] #max
          end_use_demand_hash[end_use] = demand_hash

        end

        # hash of {'consumption/demand' => {'end_use1' => {'month' => monthly_data_array, 'year' => annual_total}, ...}
        end_use_consumption_demand_hash = {}
        end_use_consumption_demand_hash['consumption'] = end_use_consumption_hash
        end_use_consumption_demand_hash['demand'] = end_use_demand_hash

        fuel_types_end_uses_consumption_demand_hash[fuel_type] = end_use_consumption_demand_hash

      end

      alternatives_hash[name] = fuel_types_end_uses_consumption_demand_hash

    end
    
    # convert hash to json and write to file
    outputs_json = alternatives_hash.to_json
    File.open('outputs.json', 'w') { |f| f << outputs_json }
    
  end

end

:pat_outputs

In [4]:
file_dir = 'C:\TOOLS\BPS-Schema\18101-PAT-SD'
file_name = 'results.json'
file_path = File.join(file_dir, file_name)

Outputs.pat_outputs(file_path)

#<File:outputs.json (closed)>